In [4]:
!pip install onnxruntime-genai
!pip install olive
!pip install huggingface-hub

  Using cached huggingface_hub-0.22.2-py3-none-any.whl.metadata (12 kB)
Using cached huggingface_hub-0.22.2-py3-none-any.whl (388 kB)


In [5]:
!huggingface-cli login --token <TOKEN>

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to C:\Users\nakersha\.cache\huggingface\token
Login successful


In [ ]:
import olive.workflows

olive.workflows.run("olive.json")

In [6]:
import onnxruntime_genai as og
import time

In [ ]:
print("Loading model...")
app_started_timestamp = time.time()

model = og.Model(f'..\..\microsoft\onnxruntime-genai\examples\python\example-models\phi2-int4-cpu-acc4')
model_loaded_timestamp  = time.time()

print("Model loaded in {:.2f} seconds".format(model_loaded_timestamp - app_started_timestamp))



Loading model...
Model loaded in 49.38 seconds


In [9]:
print("Loading tokenizer...")
tokenizer = og.Tokenizer(model)
tokenizer_stream = tokenizer.create_stream()

print("Tokenizer created")

system_prompt = "You are a helpful assistant. Answer in one sentence."
text = "What is Dilithium?"

input_tokens = tokenizer.encode(system_prompt + text)

prompt_length = len(input_tokens)

Loading tokenizer...
Tokenizer created


In [10]:
started_timestamp = time.time()

print("Creating generator ...")
params = og.GeneratorParams(model)
params.set_search_options({"do_sample": False, "max_length": 2028, "min_length": 0, "top_p": 0.9, "top_k": 40, "temperature": 1.0, "repetition_penalty": 1.0})
params.input_ids = input_tokens
generator = og.Generator(model, params)
print("Generator created")

first = True
new_tokens = []

while not generator.is_done():
    generator.compute_logits()
    generator.generate_next_token()
    if first:
        first_token_timestamp = time.time()
        first = False

    new_token = generator.get_next_tokens()[0]
    print(tokenizer_stream.decode(new_token), end="")
    new_tokens.append(new_token)

print()
run_time = time.time() - started_timestamp
print(f"Prompt tokens: {len(input_tokens)}, New tokens: {len(new_tokens)}, Time to first: {(first_token_timestamp - started_timestamp):.2f}s, New tokens per second: {len(new_tokens)/run_time:.2f} tps")


Creating generator ...
Generator created

Answer: Dilithium is a fictional substance in the Star Trek universe that is used as a propellant for spaceships.

Prompt tokens: 17, New tokens: 28, Time to first: 1.77s, New tokens per second: 4.68 tps


In [1]:
# Compare with llama.cpp.

! ..\..\..\..\ggerganov\llama.cpp\build\bin\Release\main -m ..\..\..\..\thebloke\phi-2-GGUF\phi-2.Q4_K_M.gguf --prompt "You are a helpful assistant. Answer in one sentence. What is Dilithium?"


Log start
main: build = 2647 (8228b66d)
main: built with MSVC 19.39.33523.0 for x64
main: seed  = 1712774450
llama_model_load: error loading model: llama_model_loader: failed to load model from models/7B/ggml-model-f16.gguf

llama_load_model_from_file: failed to load model
llama_init_from_gpt_params: error: failed to load model 'models/7B/ggml-model-f16.gguf'
main: error: unable to load model
